# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [3]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

In [4]:
from util_Generic import printmd, printHeader, formatLabel
from util_Generic import utl_toInteger, utl_isInteger
from util_Generic import utl_reduceMemoryUsage

In [8]:
from util_FeatureProcess import utl_splitMixedColumnValues
from util_FeatureProcess import utl_processColumns
from util_FeatureProcess import utl_cleanOnDefinition
from util_FeatureProcess import utl_cleanDataFrame

In [ ]:
from util_PCA import utl_applyPCA
from util_PCA import utl_pcaResults

In [ ]:
from CustBoostClassifier import CustBoostClassifier

In [1]:
g_saveFilenames = { 'featuresDef':             'saveGlobals/featuresDef.json',
                    'pcaMissingUnknown':       'saveGlobals/pcaMissingUnknown.sav',
                    'dummyEncode':             'saveGlobals/dummyEncode.json',
                    'categoricalToProcess':    'saveGlobals/categoricalToProcess.json',
                    'categoricalColumns':      'saveGlobals/categoricalColumns.json',
                    'highlyCorrelated':        'saveGlobals/highlyCorrelated.json',
                    'colMissing':              'saveGlobals/colMissing.json',
                    'colUnknownVals':          'saveGlobals/colUnknownVals.json' }

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [ ]:
# load in the data
for path in [ '../Arvato Project - dataSensitive', '../../data/Term2/capstone/arvato_data' ]:
    try:
        azdias    = pd.read_csv(f'{path}/Udacity_AZDIAS_052018.csv', sep=';').drop('Unnamed: 0', axis = 1)
        utl_reduceMemoryUsage('Azdias', azdias)        
        customers = pd.read_csv(f'{path}/Udacity_CUSTOMERS_052018.csv', sep=';').drop('Unnamed: 0', axis = 1)
        utl_reduceMemoryUsage('Customers', customers)
        break
    except: None

Check the number of records in both datasets and display the columns which are missing in one of the two datasets.

In [ ]:
print(f'Number of records / features in the general population dataset: {azdias.shape}')
print(f'Number of records / features in the customers dataset:          {customers.shape}')
print('')
print('Columns only available in the general population dataset:', [item for item in azdias.columns if item not in customers.columns])
print('Columns only available in the customers dataset:',          [item for item in customers.columns if item not in azdias.columns])

---
### Load and process definition for the columns based on the definition file

In the definition file there are certain values which are missing, so we will be using the ffill method in order to fill in the gaps.

We also create a flag '_isUnknown', which will help we identify the values that have the meaning 'Unknown' in the datasets.

We also check all the columns which might be a multi-level categorical variables.

In [9]:
g_featuresDef = pd.read_csv('../Arvato Project - dataSensitive/DIAS Attributes - Values 2017.csv', sep = ';', engine = 'python')
g_featuresDef.columns = ['Attribute', 'Description', 'Value', 'Meaning']

g_featuresDef['Value Converted'] = g_featuresDef['Value'].apply(lambda x: str(x).split(',')[0].strip()).apply(utl_toInteger)
g_featuresDef['Attribute']   = g_featuresDef['Attribute'].ffill()
g_featuresDef['Description'] = g_featuresDef['Description'].ffill()

for item in ['LP_STATUS_GROB', 'LP_FAMILIE_GROB']:
    v_idx = g_featuresDef[g_featuresDef['Attribute'] == item].index
    g_featuresDef.loc[v_idx, 'Meaning'] = g_featuresDef.loc[v_idx, 'Meaning'].ffill()

# Create the new dictionary containing the definition for the column
v_featuresDef = g_featuresDef.T.to_dict()   
g_featuresDef = {}
for value in v_featuresDef.values():
    v_key = value['Attribute']
    if not v_key in g_featuresDef.keys():
        g_featuresDef[v_key] = { 'Description': value['Description'].strip(),
                                 'Values':      {} }    
        
    if 'unknown' in value['Meaning']:
        g_featuresDef[v_key]['Value Unknown'] = value['Value Converted']  
    
    for item in value['Value'].split(','):  
        item = utl_toInteger(item.strip())
        if not item in g_featuresDef[v_key]['Values'].keys():
            g_featuresDef[v_key]['Values'][item] = {}

        g_featuresDef[v_key]['Values'][item]['Meaning'] = value['Meaning']
        g_featuresDef[v_key]['Values'][item]['Value Converted'] = value['Value Converted']

# Force the creation of a value UNKNOWN if none was provided in the definition file
for key in g_featuresDef.keys():
    if not 'Value Unknown' in g_featuresDef[key].keys():
        g_featuresDef[key]['Value Unknown'] = None
        
# For the mixed columns in the dataset, calculate the splitted features to be used
v_columns = [ 'ALTER_HH', 'CAMEO_DEUINTL_2015', 'PRAEGENDE_JUGENDJAHRE',
              'D19_BANKEN_DATUM',  'D19_BANKEN_ONLINE_DATUM',  'D19_BANKEN_OFFLINE_DATUM', 
              'D19_GESAMT_DATUM',  'D19_GESAMT_ONLINE_DATUM',  'D19_GESAMT_OFFLINE_DATUM',
              'D19_TELKO_DATUM',   'D19_TELKO_ONLINE_DATUM',   'D19_TELKO_OFFLINE_DATUM',
              'D19_VERSAND_DATUM', 'D19_VERSAND_ONLINE_DATUM', 'D19_VERSAND_OFFLINE_DATUM',
              'LP_LEBENSPHASE_GROB', 'LP_LEBENSPHASE_FEIN', 
              'LP_STATUS_GROB',      'LP_STATUS_FEIN', 
              'LP_FAMILIE_GROB',     'LP_FAMILIE_FEIN' ]
for column in v_columns:
    v_values = g_featuresDef[column]['Values']
    v_valuesConverted = {}
    for key, value in v_values.items():
        v_valuesConverted[value['Value Converted']] = { '00_Meaning': value['Meaning'] }
    g_featuresDef[column]['Split'] = utl_splitMixedColumnValues( p_column       = column, 
                                                                 p_values       = v_valuesConverted, 
                                                                 p_valueUnknown = g_featuresDef[column]['Value Unknown'])

We check the split for multi-level categorical variables:

In [10]:
v_data = pd.DataFrame()
for columnKey in g_featuresDef.keys():# If columns exists in the definition for a split operation, do the split
    if 'Split' in g_featuresDef[columnKey].keys():
        v_columns = {key : value['01_Columns'] for key, value in g_featuresDef[columnKey]['Split'].items()}
        v_columns = pd.DataFrame(v_columns).T
        v_columns.columns = [f'{columnKey}{item}' for item in v_columns.columns]
        v_columns.reset_index(inplace = True) 
        v_columns = v_columns.rename(columns = {'index': f'{columnKey}_MapValue'})             
        v_data = pd.concat([v_data, v_columns.head(10)], axis = 1)
display(v_data)

,ALTER_HH_MapValue,ALTER_HH_Value_0,ALTER_HH_Value_1,CAMEO_DEUINTL_2015_MapValue,CAMEO_DEUINTL_2015_Value_0,CAMEO_DEUINTL_2015_Value_1,D19_BANKEN_DATUM_MapValue,D19_BANKEN_DATUM_Value_ACTIVITY_WITHIN_MONTHS,D19_BANKEN_DATUM_Value_INCREASE,D19_BANKEN_OFFLINE_DATUM_MapValue,...,LP_STATUS_FEIN_MapValue,LP_STATUS_FEIN_Value_INCOME,LP_STATUS_FEIN_Value_OTHER,LP_STATUS_GROB_MapValue,LP_STATUS_GROB_Value_INCOME,PRAEGENDE_JUGENDJAHRE_MapValue,PRAEGENDE_JUGENDJAHRE_Value_0,PRAEGENDE_JUGENDJAHRE_Value_1,PRAEGENDE_JUGENDJAHRE_Value_2,PRAEGENDE_JUGENDJAHRE_Value_3
0,0,0.0,0.0,-1,NaN,NaN,1,1,5,1,...,1,LOW-INCOME,TYPICAL,1,LOW-INCOME,-1,-1,-1,-1,-1
1,1,0.1,0.5,11,PRE-FAMILY COUPLES - SINGLES,WEALTHY HOUSEHOLDS,2,1,4,2,...,2,LOW-INCOME,ORIENTATIONSEEKING,2,LOW-INCOME,1,40,war years,Mainstream,O+W
2,2,0.6,1.0,12,YOUNG COUPLES - CHILDREN,WEALTHY HOUSEHOLDS,3,1,3,3,...,3,LOW-INCOME,ASPIRING,3,AVERAGE EARNER,2,40,reconstruction years,Avantgarde,O+W
3,3,1.1,1.5,13,FAMILIES - SCHOOL AGE CHILDREN,WEALTHY HOUSEHOLDS,4,1,2,4,...,4,LOW-INCOME,VILLAGERS,4,AVERAGE EARNER,3,50,economic miracle,Mainstream,O+W
4,4,1.6,2.0,14,OLDER FAMILIES - MATURE COUPLES,WEALTHY HOUSEHOLDS,5,1,1,5,...,5,HIGH-INCOME,MINIMALISTIC,5,AVERAGE EARNER,4,50,milk bar / Individualisation,Avantgarde,O+W
5,5,2.1,2.5,15,ELDERS IN RETIREMENT,WEALTHY HOUSEHOLDS,6,2,0,6,...,6,INDEPENDANT,WORKERS,6,INDEPENDANT,5,60,economic miracle,Mainstream,O+W
6,6,2.6,3.0,21,PRE-FAMILY COUPLES - SINGLES,PROSPEROUS HOUSEHOLDS,7,3,0,7,...,7,HOUSEOWNER,TITLE HOLDER,7,INDEPENDANT,6,60,generation 68 / student protestors,Avantgarde,W
7,7,3.1,3.5,22,YOUNG COUPLES - CHILDREN,PROSPEROUS HOUSEHOLDS,8,4,0,8,...,8,HOUSEOWNER,NEW,8,HOUSEOWNER,7,60,opponents to the building of the Wall,Avantgarde,O
8,8,3.6,4.0,23,FAMILIES - SCHOOL AGE CHILDREN,PROSPEROUS HOUSEHOLDS,9,5,0,9,...,9,HOUSEOWNER,__NONE,9,HOUSEOWNER,8,70,family orientation,Mainstream,O+W
9,9,4.1,4.5,24,OLDER FAMILIES - MATURE COUPLES,PROSPEROUS HOUSEHOLDS,10,0,0,10,...,10,TOP EARNER,__NONE,10,TOP EARNER,9,70,peace movement,Avantgarde,O+W


----
### Analysis of categorical features

We consider that all values that have the same meaning, should have only one possible value. We will check the definition dictionary and convert all multiple values to only keep the first one.

For mixed columns we will use the definition calculated before in order to create the new splitted columns.

In [ ]:
v_azdias    = utl_cleanOnDefinition( p_label       = 'Azdias',
                                     p_data        = azdias, 
                                     p_featuresDef = g_featuresDef,
                                     p_showHeader  = True )

v_customers = utl_cleanOnDefinition( p_label       = 'Customers',
                                     p_data        = customers, 
                                     p_featuresDef = g_featuresDef,
                                     p_showHeader  = True )

----
Column 'EINGEFUEGT_AM' contains an encoding of type timestamp, so we will convert it to a timestamp and extract its components.

In [ ]:
display(pd.DataFrame(v_azdias['EINGEFUEGT_AM'].value_counts()).head())
utl_processColumns(v_azdias,    'EINGEFUEGT_AM')
utl_processColumns(v_customers, 'EINGEFUEGT_AM')
display(v_azdias.head())

----
Try to convert all columns having type object, both which can be encoded as float.

In [ ]:
for column in v_azdias.select_dtypes(include=['object']).columns:
    try:
        v_azdias[column]    = v_azdias[column].astype(np.float16)
        v_customers[column] = v_customers[column].astype(np.float16)
    except: None

----
Identify categorical like columns.

In [ ]:
v_categoryColumns    = v_azdias.drop(['LNR'], axis = 1).describe().T.reset_index()
v_categoryColumns    = v_categoryColumns[(v_categoryColumns['min'] >= -2) & (v_categoryColumns['max'] <= 10)].set_index('index')
g_categoricalColumns = v_categoryColumns.index.tolist()

----
Check columns having type 'object' and do a processing using a custom mapping for the values. Show all the columns for which a mapping **is not** defined.

In [ ]:
v_objectCols = v_azdias.select_dtypes(include=['object']).copy()  
g_categoricalToProcess = []
for column in v_objectCols.columns:    
    try:
        utl_processColumns(v_objectCols, column)
        g_categoricalToProcess.append(column)
    except Exception as err: 
        print('\n*******************************************************')
        print(f'Columns {column} could not be mapped:')    
        print(err)
        print(sorted(v_objectCols[column].astype(str).value_counts().index.tolist()))
        
g_categoricalToProcess.append('PRAEGENDE_JUGENDJAHRE_Value_0')
g_categoricalToProcess = sorted(set(g_categoricalToProcess))

g_categoricalColumns.extend(g_categoricalToProcess)
g_categoricalColumns.append('ALTERSKATEGORIE_FEIN')
g_categoricalColumns = sorted(set(g_categoricalColumns))

print('\n*******************************************************')
print('Columns for which a mapping is defined:')   
display(g_categoricalToProcess)

----
### Analysis of MISSING / UNKNOWN values

Calculate all the possible values and their distribution in both dataframes.

Missing values could follow a specific pattern, so we take into account that we can have the following situations:
   - data is **Missing Completely at Random (MCAR)** - no relationship between the missingness of the data and any values, observed or missing (nothing systematic going on)
   - data is **Missing at Random (MAR)** - we have a systematic relationship between the propensity of missing values and the observed data, but not the missing data. Whether an observation is missing has nothing to do with the missing values, but it does have to do with the values of an individual’s observed variables (eg: women are less-likely to tell their age or weight)
   - data is **Missing Not at Random (MNAR)** - there is a relationship between the propensity of a value to be missing and its values. This is a case where the people with the lowest education are missing on education.

In [ ]:
g_valuesCount = utl_getValuesCount( p_azdias      = v_azdias, 
                                    p_customers   = v_customers,
                                    p_featuresDef = g_featuresDef )

------
We start by checking for a pattern for **MISSING** values.

In [ ]:
v_null = ( g_valuesCount[g_valuesCount['Value'].isnull()]
                 .groupby(['Value %_Azdias', 'Value %_Cust'])
                 .agg({'Column Name': ['count']}) )
v_null.columns = ['Count']
display(v_null[v_null['Count'] != 1].sort_values('Count', ascending = False).head(10).reset_index())

We can see that as there are multiple cases where we can see a clear correlation between the number of missing values for the different features, so we can consider that the values are **Missing Not at Random (MNAR)**.

------
We also check for a pattern for **UNKNOWN** values.

In [ ]:
v_unknown = ( g_valuesCount[g_valuesCount['_isUnknown'] == 1]
                    .groupby(['Value %_Azdias', 'Value %_Cust'])
                    .agg({'Column Name': ['count']}) )
v_unknown.columns = ['Count']
display(v_unknown[v_unknown['Count'] != 1].sort_values('Count', ascending = False).head(10).reset_index())

We can see that as there are multiple cases where we can see a clear correlation between the number of unknown values for the different features, so we can consider that the values are **Missing Not at Random (MNAR)**.

-----
As we know that we are in a MNAR situation, we will execute a **Principal component analysis (PCA)** on both **MISSING** and **UNKNOWN** data in order to identify the missing / unknown flags with a reduced dimensionality.

In [ ]:
v_missingAzdias = v_azdias[['LNR']].copy()
v_missingCust   = v_customers[['LNR']].copy()

v_cols = v_azdias.columns.tolist()
with progressbar.ProgressBar(max_value = len(v_cols)) as bar:
    v_count = 0
    for column in v_cols:
        v_missingAzdias[column] = v_azdias[column].isnull().astype(int)
        v_missingCust[column]   = v_customers[column].isnull().astype(int)

        # In case we have only one value in both dataframes (which means that the column is always filled in) than we remove the
        # column as it will not bring any information.
        if ( v_missingAzdias[column].value_counts().shape[0] == 1
             and v_missingCust[column].value_counts().shape[0] == 1 ):
            v_missingAzdias.drop(column, axis = 1, inplace = True)
            v_missingCust.drop(column, axis = 1, inplace = True)
            
        v_count += 1
        bar.update(v_count)
v_missingAzdias.columns = [f'_isMissing_{item}' for item in v_missingAzdias.columns]
        
# Save the list of columns for which we will flag if values are missing or not        
g_colMissing = v_missingAzdias.columns.tolist()

In [ ]:
v_unknownAzdias = v_azdias[['AGER_TYP']].copy()
v_unknownCust   = v_customers[['AGER_TYP']].copy()

g_colUnknownVals = {}
for column in g_valuesCount[g_valuesCount['_isUnknown'] == 1]['Column Name'].tolist():
    g_colUnknownVals[column] = g_valuesCount[   (g_valuesCount['_isUnknown'] == 1) 
                                              & (g_valuesCount['Column Name'] == column) ]['Value'].values[0]

v_cols = g_colUnknownVals.keys()
with progressbar.ProgressBar(max_value = len(v_cols)) as bar:
    v_count = 0
    for column in v_cols:
        v_value = g_colUnknownVals[column]
        v_unknownAzdias[column] = v_azdias[column].fillna(-999).apply(lambda x: 1 if x == v_value else 0 )
        v_unknownCust[column]   = v_customers[column].fillna(-999).apply(lambda x: 1 if x == v_value else 0 )

        # In case we have only one value in both dataframes (which means that the column is always known) than we remove the
        # column as it will not bring any information.
        if ( v_unknownAzdias[column].value_counts().shape[0] == 1
             and v_unknownCust[column].value_counts().shape[0] == 1 ):
            v_unknownAzdias.drop(column, axis = 1, inplace = True)
            v_unknownCust.drop(column, axis = 1, inplace = True)
            
        v_count += 1
        bar.update(v_count)       
v_unknownAzdias.columns = [f'_isUnknown_{item}' for item in v_unknownAzdias.columns]

---
Calculate the PCA for **MISSING** and **UNKNOWN** values and show the report for the weights.

In [ ]:
v_missingUnknownAzdias = pd.concat([v_missingAzdias, v_unknownAzdias], axis = 1)
g_pcaMissingUnknown = utl_applyPCA( p_data         = v_missingUnknownAzdias,
                                    p_n_components = 60,
                                    p_ShowWeights  = True,
                                    p_ShowTop      = 6,
                                    p_figHeight    = 24 )

Based of the visualisation above, we will select **20 dimensions** for the PCA, as they capture almost 100% of the total variance for unknown and missing values. 
We do a retrain for the PCA with the selected number of components, and save the trained PCA.

In [ ]:
g_pcaMissingUnknown = utl_applyPCA( p_data         = v_missingUnknownAzdias,
                                    p_n_components = 20 )

-----------
### Fill-in missing values

We first re-encode some categorical columns.

In [ ]:
for item in g_categoricalToProcess:
    utl_processColumns(v_azdias, item)
    utl_processColumns(v_customers, item)

In [ ]:
---
Dummy encode the object columns not re-encoded above.

In [ ]:
g_dummyEncode = {}
for column in v_azdias.select_dtypes(include=['object']).columns:
    v_azdias[column] = v_azdias[column].str.replace('/', '-').str.replace(' ', '_')
    v_dummy = pd.get_dummies(v_azdias[[column]])
    g_dummyEncode[column] = v_dummy.columns.tolist()

In [ ]:
def replaceDummy(p_data, p_column, p_dummyEncode):
    if p_column not in p_data.columns: 
        for column in p_dummyEncode[p_column]:
            p_data[column] = 0
        return p_data
    
    p_data[p_column] = p_data[p_column].str.replace('/', '-').str.replace(' ', '_')
    p_data = pd.get_dummies(p_data, columns = [p_column])
    
    v_cols = [item for item in p_data.columns if (p_column in item and item not in p_dummyEncode[p_column])]
    if len(v_cols) > 0:
        p_data.drop(v_cols, axis = 1, inplace = True)
    
    for column in [item for item in p_dummyEncode[p_column] if item not in p_data.columns]:
        p_data[column] = 0
    
    return p_data
     
for column in list(g_dummyEncode.keys()):
    v_azdias    = replaceDummy( p_data        = v_azdias, 
                                p_column      = column, 
                                p_dummyEncode = g_dummyEncode )
    v_customers = replaceDummy( p_data        = v_customers, 
                                p_column      = column, 
                                p_dummyEncode = g_dummyEncode )

---
Fill in columns that are categorical with their unknown value or -2.

In [ ]:
with progressbar.ProgressBar(max_value = len(v_cols)) as bar:
    v_count = 0
    for column in g_categoricalColumns:
        v_azdias[column]    = v_azdias[column].fillna(-2).astype(int)
        v_customers[column] = v_customers[column].fillna(-2).astype(int)
        v_count += 1
        bar.update(v_count)

---
Check the columns that still have empty values.

In [ ]:
v_dummyEncode = []
for value in g_dummyEncode.values():
    v_dummyEncode.extend(value)
display(sorted([item for item in v_azdias.columns if (item not in g_categoricalColumns and item not in v_dummyEncode)]))

In [ ]:
v_missingAzdias = pd.DataFrame({'Null Value': 0}, index = ['LNR'])
v_missingCust   = pd.DataFrame({'Null Value': 0}, index = ['LNR'])
for column in v_azdias.columns:
    v_missingAzdias.loc[column, 'Null Value'] = v_azdias[column].isnull().sum()
    v_missingCust.loc[column, 'Null Value']   = v_customers[column].isnull().sum()
v_missing = ( v_missingAzdias[v_missingAzdias['Null Value'] > 0]
                 .merge( v_missingCust[v_missingCust['Null Value'] > 0], 
                         how = 'outer', left_index = True, right_index = True,
                         suffixes = ('_Azdias', '_Cust') ) )
display(v_missing)

-------------
### Check for highly correlated features

In [ ]:
# Create correlation matrix
v_corr_matrix = v_azdias.drop('LNR', axis = 1).corr().abs()

# Select upper triangle of correlation matrix
v_upper = v_corr_matrix.where(np.triu(np.ones(v_corr_matrix.shape), k = 1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
v_to_drop = [column for column in v_upper.columns if any(v_upper[column] > 0.95)]

for column in v_to_drop:
    v_value = v_corr_matrix.drop(column, axis = 1).loc[column, :]
    v_value = v_value[v_value > 0.97]
    if v_value.shape[0] > 0: display(pd.DataFrame(v_value))

In [ ]:
g_highlyCorrelated = [ "DSL_FLAG", "ALTER_HH_Value_1", "EINGEFUEGT_AM_Year", "EINGEFUEGT_AM_Month", "EINGEFUEGT_AM_Quarter", 
                       "EINGEFUEGT_AM_Hour", "EINGEFUEGT_AM_Minute", "EINGEFUEGT_AM_Second", "EINGEFUEGT_AM_WeekOfYear", 
                       "LP_LEBENSPHASE_GROB_Value_FAMILY", "LP_LEBENSPHASE_FEIN_Value_FAMILY", "ANZ_TITEL", 
                       "ANZ_STATISTISCHE_HAUSHALTE", "PLZ8_HHZ", "PLZ8_GBZ", "KBA13_SEG_GROSSRAUMVANS", "KBA13_KMH_211", 
                       "KBA13_ALTERHALTER_61", "KBA13_ALTERHALTER_60", "KBA13_ALTERHALTER_45", "KBA13_ALTERHALTER_30", 
                       "KBA05_KRSKLEIN", "KBA05_HERST3", "KBA05_HERST2" ]

--------
### Save all global variables / models for further use.

In [ ]:
joblib.dump(g_pcaMissingUnknown, g_saveFilenames['pcaMissingUnknown'])

with open(g_saveFilenames['featuresDef'], 'w') as outfile:
    json.dump(g_featuresDef, outfile)
    
with open(g_saveFilenames['categoricalColumns'], 'w') as outfile:
    json.dump(g_categoricalColumns, outfile)
    
with open(g_saveFilenames['categoricalToProcess'], 'w') as outfile:
    json.dump(g_categoricalToProcess, outfile)
    
with open(g_saveFilenames['dummyEncode'], 'w') as outfile:
    json.dump(g_dummyEncode, outfile)
    
with open(g_saveFilenames['highlyCorrelated'], 'w') as outfile:
    json.dump(g_highlyCorrelated, outfile)
    
with open(g_saveFilenames['colMissing'], 'w') as outfile:
    json.dump(g_colMissing, outfile)
    
with open(g_saveFilenames['colUnknownVals'], 'w') as outfile:
    json.dump(g_colUnknownVals, outfile)

-----------
-------------
----------

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

------
### Clean both datasets

Based on finding above, we will create a cleaning procedure that we will apply on all future datasets having the same format.

For the cleaning of datasets, we will execute the following operations:
   1. **clean data based on the definition** - only one possible value will be kept when there were multiple values specififed for the same row in the definitions
   2. re-encode categorical features
   3. apply the Principal Component Analysis for the **missing values**, by using the trained PCA from above
   4. apply the Principal Component Analysis for the **unknow values**, by using the trained PCA from above
   5. fill in categorical columns with -2
   6. drop features identified as highly correlated
   7. calculate columns to be imputed later
   8. calculate columns to be scaled later

In [ ]:
# Load variables saved earlier
v_pcaMissingUnknown = joblib.load(g_saveFilenames['pcaMissingUnknown'])

with open(g_saveFilenames['featuresDef'], 'r') as inFile:
    v_featuresDef = json.load(inFile)
    
with open(g_saveFilenames['categoricalColumns'], 'r') as inFile:
    v_categoricalColumns = json.load(inFile)
    
with open(g_saveFilenames['categoricalToProcess'], 'r') as inFile:
    v_categoricalToProcess = json.load(inFile)
    
with open(g_saveFilenames['dummyEncode'], 'r') as inFile:
    v_dummyEncode = json.load(inFile)
    
with open(g_saveFilenames['highlyCorrelated'], 'r') as inFile:
    v_highlyCorrelated = json.load(inFile)
    
with open(g_saveFilenames['colMissing'], 'r') as inFile:
    v_colMissing = json.load(inFile)
    
with open(g_saveFilenames['colUnknownVals'], 'r') as inFile:
    v_colUnknownVals = json.load(inFile)
    
v_azdias, v_missing, v_scaleColumns = utl_cleanDataFrame( p_label                 = 'Azdias', 
                                                          p_data                  = azdias, 
                                                          p_featuresDef           = v_featuresDef, 
                                                          p_categoricalColumns    = v_categoricalColumns, 
                                                          p_categoricalToProcess  = v_categoricalToProcess, 
                                                          p_dummyEncode           = v_dummyEncode, 
                                                          p_highlyCorrelated      = v_highlyCorrelated, 
                                                          p_pcaMissingUnknown     = v_pcaMissingUnknown, 
                                                          p_colMissing            = v_colMissing, 
                                                          p_colUnknownVals        = v_colUnknownVals,
                                                          p_ignoreScale           = [ 'LNR' ] )
    
v_customers, _, _ = utl_cleanDataFrame( p_label                 = 'Customers', 
                                        p_data                  = customers, 
                                        p_featuresDef           = v_featuresDef, 
                                        p_categoricalColumns    = v_categoricalColumns, 
                                        p_categoricalToProcess  = v_categoricalToProcess, 
                                        p_dummyEncode           = v_dummyEncode, 
                                        p_highlyCorrelated      = v_highlyCorrelated, 
                                        p_pcaMissingUnknown     = v_pcaMissingUnknown, 
                                        p_colMissing            = v_colMissing, 
                                        p_colUnknownVals        = v_colUnknownVals,
                                        p_ignoreScale           = [ 'LNR' ] )

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
for path in [ '../Arvato Project - dataSensitive', '../../data/Term2/capstone/arvato_data' ]:
    try:
        mailout_train = pd.read_csv(f'{path}/Udacity_MAILOUT_052018_TRAIN.csv', sep=';').drop('Unnamed: 0', axis = 1)
        break
    except: None
        
# Load variables saved earlier
v_pcaMissingUnknown = joblib.load(g_saveFilenames['pcaMissingUnknown'])

with open(g_saveFilenames['featuresDef'], 'r') as inFile:
    v_featuresDef = json.load(inFile)
    
with open(g_saveFilenames['categoricalColumns'], 'r') as inFile:
    v_categoricalColumns = json.load(inFile)
    
with open(g_saveFilenames['categoricalToProcess'], 'r') as inFile:
    v_categoricalToProcess = json.load(inFile)
    
with open(g_saveFilenames['dummyEncode'], 'r') as inFile:
    v_dummyEncode = json.load(inFile)
    
with open(g_saveFilenames['highlyCorrelated'], 'r') as inFile:
    v_highlyCorrelated = json.load(inFile)
    
with open(g_saveFilenames['colMissing'], 'r') as inFile:
    v_colMissing = json.load(inFile)
    
with open(g_saveFilenames['colUnknownVals'], 'r') as inFile:
    v_colUnknownVals = json.load(inFile)
    
v_mailout_train, v_missing, v_scaleColumns = utl_cleanDataFrame( p_label                 = 'Train Dataset', 
                                                                 p_data                  = mailout_train, 
                                                                 p_featuresDef           = v_featuresDef, 
                                                                 p_categoricalColumns    = v_categoricalColumns, 
                                                                 p_categoricalToProcess  = v_categoricalToProcess, 
                                                                 p_dummyEncode           = v_dummyEncode, 
                                                                 p_highlyCorrelated      = v_highlyCorrelated, 
                                                                 p_pcaMissingUnknown     = v_pcaMissingUnknown, 
                                                                 p_colMissing            = v_colMissing, 
                                                                 p_colUnknownVals        = v_colUnknownVals,
                                                                 p_ignoreScale           = [ 'LNR', 'RESPONSE' ] )

------
### Split in train and test datasets

In [ ]:
v_features = v_mailout_train.drop(['RESPONSE', 'LNR'], axis = 1)
v_target   = v_mailout_train['RESPONSE']
X_trainAll, X_testAll, y_trainAll, y_testAll = train_test_split( v_features, 
                                                                 v_target, 
                                                                 test_size = .10, 
                                                                 random_state = 42,
                                                                 stratify = v_target )

------
### Define the weight for the positive class.

In [ ]:
print('Positive weight for the complete target: ', round(float(np.sum(v_target == 0)) / np.sum(v_target == 1), 3))
print('Positive weight for the train dataset:   ', round(float(np.sum(y_trainAll == 0)) / np.sum(y_trainAll == 1), 3))
print('Positive weight for the test dataset:    ', round(float(np.sum(y_testAll == 0)) / np.sum(y_testAll == 1), 3))
v_posWeight = 79.75

------
### Bayesian search for parameters for a LGB model with all features, no imputing, no scaling

In [ ]:
v_model = CustBoostClassifier( p_modelType          = 'lightgbm',
                               p_modelName          = 'LGB_AllFeatures_NI_NS',
                               X_train              = X_trainAll.copy(),
                               y_train              = y_trainAll.copy(),
                               p_selected_features  = [],
                               p_imputeColumns      = [], 
                               p_scaleColumns       = [], 
                               p_cvSplits           = 10,
                               p_random_state       = 2013 )   
v_bayes_trials = g_model.bayesianSearchLGB( p_posWeight  = v_posWeight,
                                            p_max_eval   = 600,
                                            p_topTrials  = 30,
                                            X_test       = X_testAll, 
                                            y_test       = y_testAll )

------
### Bayesian search for parameters for a LGB model with selected features, no imputing, no scaling

In [ ]:
v_selected_features = []
v_model = CustBoostClassifier( p_modelType          = 'lightgbm',
                               p_modelName          = 'LGB_AllFeatures_NI_NS',
                               X_train              = X_trainAll.copy(),
                               y_train              = y_trainAll.copy(),
                               p_selected_features  = v_selected_features,
                               p_imputeColumns      = [], 
                               p_scaleColumns       = [], 
                               p_cvSplits           = 10,
                               p_random_state       = 2013 )   
v_bayes_trials = g_model.bayesianSearchLGB( p_posWeight  = v_posWeight,
                                            p_max_eval   = 600,
                                            p_topTrials  = 30,
                                            X_test       = X_testAll, 
                                            y_test       = y_testAll )

------
### Bayesian search for parameters for a LGB model with selected features, imputing and scaling

In [ ]:
v_selected_features = []
v_model = CustBoostClassifier( p_modelType          = 'lightgbm',
                               p_modelName          = 'LGB_AllFeatures_NI_NS',
                               X_train              = X_trainAll.copy(),
                               y_train              = y_trainAll.copy(),
                               p_selected_features  = v_selected_features,
                               p_imputeColumns      = v_missing, 
                               p_scaleColumns       = v_scaleColumns, 
                               p_cvSplits           = 10,
                               p_random_state       = 2013 )   
v_bayes_trials = g_model.bayesianSearchLGB( p_posWeight  = v_posWeight,
                                            p_max_eval   = 600,
                                            p_topTrials  = 30,
                                            X_test       = X_testAll, 
                                            y_test       = y_testAll )

-----
### Make predictions with stack models

In [ ]:
v_model = CustBoostClassifier( p_modelType          = 'lightgbm',
                               p_random_state       = 2013 )   
v_model.predictStack( p_folder   = [ '' ],
                      X_data     = X_testAll, 
                      y_data     = y_testAll, 
                      p_showPlot = True )

-----
----
----

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [13]:
for path in [ '../Arvato Project - dataSensitive', '../../data/Term2/capstone/arvato_data' ]:
    try:
        mailout_test = pd.read_csv(f'{path}/Udacity_MAILOUT_052018_TEST.csv', sep=';').drop('Unnamed: 0', axis = 1)
        break
    except: None
        
# Load variables saved earlier
v_pcaMissingUnknown = joblib.load(g_saveFilenames['pcaMissingUnknown'])

with open(g_saveFilenames['featuresDef'], 'r') as inFile:
    v_featuresDef = json.load(inFile)
    
with open(g_saveFilenames['categoricalColumns'], 'r') as inFile:
    v_categoricalColumns = json.load(inFile)
    
with open(g_saveFilenames['categoricalToProcess'], 'r') as inFile:
    v_categoricalToProcess = json.load(inFile)
    
with open(g_saveFilenames['dummyEncode'], 'r') as inFile:
    v_dummyEncode = json.load(inFile)
    
with open(g_saveFilenames['highlyCorrelated'], 'r') as inFile:
    v_highlyCorrelated = json.load(inFile)
    
with open(g_saveFilenames['colMissing'], 'r') as inFile:
    v_colMissing = json.load(inFile)
    
with open(g_saveFilenames['colUnknownVals'], 'r') as inFile:
    v_colUnknownVals = json.load(inFile)
    
v_mailout_test, v_missing, v_scaleColumns = utl_cleanDataFrame( p_label                 = 'Test Dataset', 
                                                                p_data                  = mailout_test, 
                                                                p_featuresDef           = v_featuresDef, 
                                                                p_categoricalColumns    = v_categoricalColumns, 
                                                                p_categoricalToProcess  = v_categoricalToProcess, 
                                                                p_dummyEncode           = v_dummyEncode, 
                                                                p_highlyCorrelated      = v_highlyCorrelated, 
                                                                p_pcaMissingUnknown     = v_pcaMissingUnknown, 
                                                                p_colMissing            = v_colMissing, 
                                                                p_colUnknownVals        = v_colUnknownVals,
                                                                p_ignoreScale           = [ 'LNR', 'RESPONSE' ] )

D:\___winApp\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
